# Core Tutorial

This script will introduce you to the basics of time series handling with pynapple.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pynapple as nap

## Time series object

Let's create a Tsd object with fake data. In this case, every time point is 1 second apart. A Tsd object is a wrapper of pandas series.

In [4]:
tsd = nap.Tsd(t = np.arange(100), d = np.random.rand(100), time_units = 's')

print(tsd)

Time (s)
0.0     0.589780
1.0     0.326939
2.0     0.102468
3.0     0.421600
4.0     0.531665
          ...   
95.0    0.660078
96.0    0.965494
97.0    0.122592
98.0    0.631948
99.0    0.419832
Length: 100, dtype: float64


While the tsd object appears in second, it actually holds the values in microseconds by default. It is possible to switch between seconds, milliseconds and microseconds. Note that when using *as_units*, the returned object is a simple pandas series.

In [8]:
print(tsd.as_units('ms'))
print(tsd.as_units('us'))

Time (ms)
0.0        0.589780
1000.0     0.326939
2000.0     0.102468
3000.0     0.421600
4000.0     0.531665
             ...   
95000.0    0.660078
96000.0    0.965494
97000.0    0.122592
98000.0    0.631948
99000.0    0.419832
Length: 100, dtype: float64
Time (us)
0           0.589780
1000000     0.326939
2000000     0.102468
3000000     0.421600
4000000     0.531665
              ...   
95000000    0.660078
96000000    0.965494
97000000    0.122592
98000000    0.631948
99000000    0.419832
Length: 100, dtype: float64


If only timestamps are available, for example spike times, we can construct a Ts object which holds only times. In this case, we generate 10 random spike times between 0 and 100 ms.

In [10]:
ts = nap.Ts(t = np.sort(np.random.uniform(0, 100, 10)), time_units = 'ms')

print(ts)

Time (s)
0.000053   NaN
0.018750   NaN
0.022492   NaN
0.031866   NaN
0.037524   NaN
0.043755   NaN
0.045585   NaN
0.069245   NaN
0.077722   NaN
0.088523   NaN
dtype: float64


If the time series contains multiple columns, we can use a TsdFrame.

In [12]:
tsdframe = nap.TsdFrame(t = np.arange(100), d = np.random.rand(100,3), time_units = 's', columns = ['a', 'b', 'c'])

print(tsdframe)

                 a         b         c
Time (s)                              
0.0       0.248641  0.792004  0.089255
1.0       0.502755  0.807717  0.912538
2.0       0.761824  0.022373  0.210368
3.0       0.881189  0.666426  0.316277
4.0       0.633466  0.178083  0.965424
...            ...       ...       ...
95.0      0.102830  0.614337  0.158703
96.0      0.356926  0.798900  0.405869
97.0      0.084771  0.058427  0.087754
98.0      0.269665  0.411934  0.944591
99.0      0.358653  0.409465  0.112126

[100 rows x 3 columns]


## Interval Sets object

The [IntervalSet](https://peyrachelab.github.io/pynapple/core.interval_set/) object stores multiple epochs with a common time units. It can then be used to restrict time series to this particular set of epochs.

In [16]:
epochs = nap.IntervalSet(start = [0, 10], end = [5, 15], time_units = 's')

new_tsd = tsd.restrict(epochs)

print(epochs)
print('\n')
print(new_tsd)

   start   end
0    0.0   5.0
1   10.0  15.0


Time (s)
1.0     0.326939
2.0     0.102468
3.0     0.421600
4.0     0.531665
5.0     0.069164
11.0    0.559547
12.0    0.439060
13.0    0.252566
14.0    0.934285
15.0    0.415332
dtype: float64


Multiple operations are available for IntervalSet